In [1]:
!pip install huggingface_hub
!pip install transformers datasets torch
!pip install --upgrade torch
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch \
    torchdata \
    transformers[torch] \
    evaluate \
    rouge_score \
    loralib \
    datasets \


!pip install 'accelerate>=0.26.0' --quiet
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninsta

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
nvidiagpu = !nvidia-smi
nvidiagpu

['Thu Dec 19 11:27:11 2024       ',
 '+---------------------------------------------------------------------------------------+',
 '| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |',
 '|-----------------------------------------+----------------------+----------------------+',
 '| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |',
 '| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |',
 '|                                         |                      |               MIG M. |',
 '|=========================================+======================+======================|',
 '|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |',
 '| N/A   68C    P8              16W /  72W |      4MiB / 23034MiB |      0%      Default |',
 '|                                         |                      |                  N/A |',
 '+---------------------

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.autograd.set_detect_anomaly(True)
f"Using device: {device}"

'Using device: cuda'

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset

dataset = load_dataset("super_glue", "rte", trust_remote_code=True)
dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

super_glue.py:   0%|          | 0.00/30.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 277
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 3000
    })
})

In [8]:
trainData = dataset['train']
testData = dataset['test']
combined_texts = [f"{p} {h}" for p, h in zip(trainData['premise'], trainData['hypothesis'])]
combined_texts

['No Weapons of Mass Destruction Found in Iraq Yet. Weapons of Mass Destruction Found in Iraq.',
 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI. Pope Benedict XVI is the new leader of the Roman Catholic Church.',
 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients. Herceptin can be used to treat breast cancer.',
 'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment. The previous name of Ho Chi Minh City was Saigon.',
 "A man is due in court later charged with the murder 26 years ago of a teenager whose case w

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [10]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()

    return \
        f"Trainable model parameters: {trainable_model_params}\n" +\
        f"All model parameters: {all_model_params}\n" +\
        f"Percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

Trainable model parameters: 124439808
All model parameters: 124439808
Percentage of trainable model parameters: 100.00%


In [11]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(example):
    prompt = [f"Premise: {premise} Hypothesis: {hypothesis}" for premise, hypothesis in zip(example['premise'], example['hypothesis'])]
    encoding = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=512)

    input_ids = encoding.input_ids
    labels = input_ids.clone()  # Sử dụng input_ids làm labels cho causal language model

    # Chỉnh sửa nhãn: GPT-2 sử dụng "causal language modeling" nên nhãn là chính nó.
    # Đảm bảo rằng nếu có padding, sẽ không có sự ảnh hưởng đến loss
    labels[labels == tokenizer.pad_token_id] = -100

    return {
        'input_ids': input_ids.squeeze(),  # Loại bỏ chiều dư thừa
        'labels': labels.squeeze()          # Loại bỏ chiều dư thừa ở labels
    }

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['idx', 'premise', 'hypothesis', 'label'])



Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [12]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (2490, 2)
Validation: (277, 2)
Test: (3000, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 277
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3000
    })
})


In [13]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [14]:
output_dir = "./results"
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_steps=100,
    per_device_train_batch_size=2,   # Điều chỉnh batch size nếu gặp vấn đề bộ nhớ
    per_device_eval_batch_size=2,    # Điều chỉnh batch size cho đánh giá
    logging_dir='./logs',            # Thư mục ghi log
    evaluation_strategy="epoch",     # Đánh giá sau mỗi epoch
    save_strategy="epoch",           # Lưu mô hình sau mỗi epoch
    fp16=True,                       # Bật mixed precision
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-836ed3ed4982>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
!pip install plotly

In [16]:
log_history = trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,3.031800,2.826963
2,2.837000,2.815758
3,2.730900,2.820941
4,2.599700,2.832351
5,2.516900,2.852902
6,2.414800,2.865456
7,2.299800,2.892920
8,2.236200,2.914765
9,2.134700,2.940462
10,2.036800,2.968353


In [23]:
import plotly.graph_objects as go

# Extract the log history from the trainer
log_history = trainer.state.log_history

# Filter epoch-level training and evaluation losses
train_losses = [log['loss'] for log in log_history if 'loss' in log and 'epoch' in log]
eval_losses = [log['eval_loss'] for log in log_history if 'eval_loss' in log]
epochs_train = [log['epoch'] for log in log_history if 'loss' in log and 'epoch' in log]
epochs_eval = [log['epoch'] for log in log_history if 'eval_loss' in log]

# Create the plot using Plotly
fig = go.Figure()

# Add training loss trace
fig.add_trace(go.Scatter(
    x=epochs_train,
    y=train_losses,
    mode='lines+markers',
    name='Train Loss',
    line=dict(color='blue'),
    marker=dict(symbol='circle')
))

# Add evaluation loss trace
fig.add_trace(go.Scatter(
    x=epochs_eval,
    y=eval_losses,
    mode='lines+markers',
    name='Eval Loss',
    line=dict(color='red'),
    marker=dict(symbol='x')
))

# Customize layout
fig.update_layout(
    title='Training and Evaluation Loss Per Epoch',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    legend_title='Loss Type',
    template='plotly_dark',
    showlegend=True
)

# Show the plot
fig.show()
